In [ ]:
import json
from py2neo import Graph, Node, Relationship
#from py2neo.Graph import database 

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")
# graph = Graph('bolt://localhost:7687', bolt=True)

#graph.delete_all()

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

In [ ]:
##import the publications where lang = 'en' and publisher = "Science" or "Nature" in year 2008
import pandas as pd
import time
print("load english science and nature publication into df")
start_time = time.time()
query = """
MATCH (n:Quanta) WHERE n.lang = 'en' AND ( n.venue = 'Science' OR n.venue = 'Nature') AND n.year =2008 AND EXISTS(n.fos)
RETURN 
n.venue as venue,
n.pageRank_2018 as PR_2018,
n.pageRank_2008 as PR_2008,
n.fos as fos,
n.title as title,
n.keywords as keywords,
n.publisher as publisher
ORDER BY n.pageRank_2018 DESC
"""
#n.keywords as keywords
dfs_2008_test = graph.run(query).to_data_frame()
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))
#dfs_2008_test

In [ ]:
top_range = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import itertools

dfs_2008_test_copy = dfs_2008_test.copy()
start_time = time.time()
fos_list = dfs_2008_test_copy["fos"].tolist()
fos_list = [[] if v is None else v for v in fos_list]


dfs_2008_test_copy.head()
#Replace original fos with updated fos 
dfs_2008_test_copy['fos'] = pd.Series(fos_list).values
dfs_2008_test

mlb = MultiLabelBinarizer()
X = mlb.fit_transform(dfs_2008_test_copy.fos)
dfs_2008_test_copy = dfs_2008_test_copy.join(pd.DataFrame(X, columns=mlb.classes_))

#del fos in the df

del dfs_2008_test_copy['fos']
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))
dfs_2008_test_copy

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(dfs_2008_test.title).toarray()
#labels = df.category_id
print(features.shape)

print(len(tfidf.get_feature_names()))
print(type(features))

title_feature_name = tfidf.get_feature_names()
count_featurename = len(title_feature_name)
for i in range(count_featurename):
    column_name = 'title_'+title_feature_name[i]
    dfs_2008_test_copy[column_name] = pd.Series(features[:,i]).values

dfs_2008_test_copy

In [ ]:
dfDummies = pd.get_dummies(dfs_2008_test_copy['venue'], prefix = 'venue')
dfs_2008_test_copy = pd.concat([dfs_2008_test_copy, dfDummies], axis=1)
del dfs_2008_test_copy['venue']
del dfs_2008_test_copy['keywords']
del dfs_2008_test_copy['publisher']
del dfs_2008_test_copy['PR_2008']
del dfs_2008_test_copy['title']
del dfs_2008_test_copy['PR_2018']
dfs_2008_test_copy